In [1]:
import pandas as pd

pd.set_option('display.max_columns',None)

In [2]:
df = pd.read_excel('data.xlsx')
df.head()

,Customer ID,Tenure Months,Location,Device Class,Games Product,Music Product,Education Product,Call Center,Video Product,Use MyApp,Payment Method,Monthly Purchase (Thou. IDR),Churn Label,Longitude,Latitude,CLTV (Predicted Thou. IDR)
0,0,2,Jakarta,Mid End,Yes,Yes,No,No,No,No,Digital Wallet,70.005,Yes,106.816666,-6.2,4210.7
1,1,2,Jakarta,High End,No,No,No,No,No,No,Pulsa,91.910,Yes,106.816666,-6.2,3511.3
2,2,8,Jakarta,High End,No,No,Yes,No,Yes,Yes,Pulsa,129.545,Yes,106.816666,-6.2,6983.6
3,3,28,Jakarta,High End,No,No,Yes,Yes,Yes,Yes,Pulsa,136.240,Yes,106.816666,-6.2,6503.9
4,4,49,Jakarta,High End,No,Yes,Yes,No,Yes,Yes,Debit,134.810,Yes,106.816666,-6.2,6942.0


In [3]:
df.drop(columns=['Longitude','Latitude'],inplace=True)

In [10]:

products = ["Games Product", "Music Product", "Education Product", "Video Product", "Call Center", "Use MyApp"]
items = df[products] 
items

,Games Product,Music Product,Education Product,Video Product,Call Center,Use MyApp
0,Yes,Yes,No,No,No,No
1,No,No,No,No,No,No
2,No,No,Yes,Yes,No,Yes
3,No,No,Yes,Yes,Yes,Yes
4,No,Yes,Yes,Yes,No,Yes
...,...,...,...,...,...,...
7038,No internet service,No internet service,No internet service,No internet service,No,No internet service
7039,Yes,No,Yes,Yes,Yes,Yes
7040,No,Yes,Yes,Yes,No,Yes
7041,Yes,No,No,No,No,No


In [11]:
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)
import pandas as pd


data_train = pd.read_csv("iris.csv")
data_test = pd.read_csv("iris.csv")

txns_train = TransactionDB.from_DataFrame(data_train)
txns_test = TransactionDB.from_DataFrame(data_test)

# get the best association rules
rules = top_rules(txns_train.string_representation)

# convert them to class association rules
cars = createCARs(rules)

classifier = M1Algorithm(cars, txns_train).build()
# classifier = M2Algorithm(cars, txns_train).build()

accuracy = classifier.test_transactions(txns_test)


FileNotFoundError: [Errno 2] No such file or directory: 'iris.csv'

In [8]:
X

,Games Product,Music Product,Education Product,Video Product,Call Center,Use MyApp
0,Yes,Yes,No,No,No,No
1,No,No,No,No,No,No
2,No,No,Yes,Yes,No,Yes
3,No,No,Yes,Yes,Yes,Yes
4,No,Yes,Yes,Yes,No,Yes
...,...,...,...,...,...,...
7038,No internet service,No internet service,No internet service,No internet service,No,No internet service
7039,Yes,No,Yes,Yes,Yes,Yes
7040,No,Yes,Yes,Yes,No,Yes
7041,Yes,No,No,No,No,No


In [4]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

def frozenset_to_string(fset):
    elements = ', '.join(map(str, fset))
    return f"({elements})"

def index(min_support, min_confidence):
    df = pd.read_excel("data.xlsx")
    products = ["Games Product", "Music Product", "Education Product", "Video Product", "Call Center", "Use MyApp"]
    X = df.copy()[products]

    transactions = list()
    for i in range(len(X)):
        x = X.iloc[i]

        product_used = list()
        for product_name, usage in zip(x.index, x):
            if usage == "Yes":
                product_used.append(product_name)
        transactions.append(product_used)

    encoder = TransactionEncoder()
    one_hot = encoder.fit(transactions).transform(transactions)
    one_hot_df = pd.DataFrame(one_hot, columns=encoder.columns_)

    fpgrowth_result = fpgrowth(one_hot_df, min_support=min_support, use_colnames=True, verbose=1)
    df_ar = association_rules(fpgrowth_result, min_threshold=min_confidence)

    df_ar['antecedents'] = df_ar['antecedents'].apply(lambda x: frozenset_to_string(x))
    df_ar['consequents'] = df_ar['consequents'].apply(lambda x: frozenset_to_string(x))

    # Group by location
    location_df = df.groupby('Location').apply(lambda x: index_for_location(x, min_support, min_confidence))
    
    return location_df

def index_for_location(df, min_support, min_confidence):
    products = ["Games Product", "Music Product", "Education Product", "Video Product", "Call Center", "Use MyApp"]

    transactions = list()
    for i in range(len(df)):
        x = df.iloc[i]

        product_used = list()
        for product_name, usage in zip(x.index, x):
            if usage == "Yes":
                product_used.append(product_name)
        transactions.append(product_used)

    encoder = TransactionEncoder()
    one_hot = encoder.fit(transactions).transform(transactions)
    one_hot_df = pd.DataFrame(one_hot, columns=encoder.columns_)

    fpgrowth_result = fpgrowth(one_hot_df, min_support=min_support, use_colnames=True, verbose=1)
    df_ar = association_rules(fpgrowth_result, min_threshold=min_confidence)

    df_ar['antecedents'] = df_ar['antecedents'].apply(lambda x: frozenset_to_string(x))
    df_ar['consequents'] = df_ar['consequents'].apply(lambda x: frozenset_to_string(x))
    
    return df_ar


In [5]:
min_support = 0.2  # Set your desired minimum support
min_confidence = 0.5  # Set your desired minimum confidence

result_df = index(min_support, min_confidence)

print(result_df)


6 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (Music Product)
0 itemset(s) from tree conditioned on items (Games Product)
0 itemset(s) from tree conditioned on items (Use MyApp)
1 itemset(s) from tree conditioned on items (Video Product)
2 itemset(s) from tree conditioned on items (Education Product)
0 itemset(s) from tree conditioned on items (Education Product, Use MyApp)
0 itemset(s) from tree conditioned on items (Education Product, Video Product)
0 itemset(s) from tree conditioned on items (Call Center)
7 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (Games Product)
0 itemset(s) from tree conditioned on items (Churn Label)
0 itemset(s) from tree conditioned on items (Use MyApp)
0 itemset(s) from tree conditioned on items (Music Product)
2 itemset(s) from tree conditioned on items (Education Product)
0 itemset(s) from tree conditioned on items (Education Product, Use MyApp)
0 itemset(s) from tre